In [1]:
from pyspark.sql import SparkSession

# Создаем сессию Spark
spark = SparkSession.builder \
    .appName("MyApp") \
    .getOrCreate()

In [2]:
from pyspark.sql.types import *
import pyspark.sql.functions as F

# Programmatic way to define a schema 
fire_schema = StructType([StructField('CallNumber', IntegerType(), True),
                     StructField('UnitID', StringType(), True),
                     StructField('IncidentNumber', IntegerType(), True),
                     StructField('CallType', StringType(), True),                  
                     StructField('CallDate', StringType(), True),      
                     StructField('WatchDate', StringType(), True),
                     StructField('CallFinalDisposition', StringType(), True),
                     StructField('AvailableDtTm', StringType(), True),
                     StructField('Address', StringType(), True),       
                     StructField('City', StringType(), True),       
                     StructField('Zipcode', IntegerType(), True),       
                     StructField('Battalion', StringType(), True),                 
                     StructField('StationArea', StringType(), True),       
                     StructField('Box', StringType(), True),       
                     StructField('OriginalPriority', StringType(), True),       
                     StructField('Priority', StringType(), True),       
                     StructField('FinalPriority', IntegerType(), True),       
                     StructField('ALSUnit', BooleanType(), True),       
                     StructField('CallTypeGroup', StringType(), True),
                     StructField('NumAlarms', IntegerType(), True),
                     StructField('UnitType', StringType(), True),
                     StructField('UnitSequenceInCallDispatch', IntegerType(), True),
                     StructField('FirePreventionDistrict', StringType(), True),
                     StructField('SupervisorDistrict', StringType(), True),
                     StructField('Neighborhood', StringType(), True),
                     StructField('Location', StringType(), True),
                     StructField('RowID', StringType(), True),
                     StructField('Delay', FloatType(), True)])

# Use the DataFrameReader interface to read a CSV file
sf_fire_file = "sf-fire-calls.csv"
fire_df = spark.read.csv(sf_fire_file, header=True, schema=fire_schema)

# Cache the DataFrame since we will be performing some operations on it.
fire_df.cache()
fire_df.count()

fire_df.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [3]:
#display(fire_df.limit(5))
fire_df.show(5, vertical=True)

-RECORD 0------------------------------------------
 CallNumber                 | 20110016             
 UnitID                     | T13                  
 IncidentNumber             | 2003235              
 CallType                   | Structure Fire       
 CallDate                   | 01/11/2002           
 WatchDate                  | 01/10/2002           
 CallFinalDisposition       | Other                
 AvailableDtTm              | 01/11/2002 01:51:... 
 Address                    | 2000 Block of CAL... 
 City                       | SF                   
 Zipcode                    | 94109                
 Battalion                  | B04                  
 StationArea                | 38                   
 Box                        | 3362                 
 OriginalPriority           | 3                    
 Priority                   | 3                    
 FinalPriority              | 3                    
 ALSUnit                    | false                
 CallTypeGro

In [4]:
# Filter out "Medical Incident" call types

# Note that filter() and where() methods on the DataFrame are similar. Check relevant documentation for their respective argument types.
few_fire_df = (fire_df
               .select("IncidentNumber", "AvailableDtTm", "CallType")
               .where(F.col("CallType") != "Medical Incident"))
few_fire_df.show(1, vertical=True)

-RECORD 0------------------------------
 IncidentNumber | 2003235              
 AvailableDtTm  | 01/11/2002 01:51:... 
 CallType       | Structure Fire       
only showing top 1 row



In [5]:
# return number of distinct types of calls using countDistinct()
from pyspark.sql.functions import *
(fire_df
  .select("CallType")
  .where(F.col("CallType").isNotNull())
  .agg(F.countDistinct("CallType").alias("DistinctCallTypes"))
  .show())

(fire_df
  .select("CallType")
  .where(F.col("CallType").isNotNull())
  .distinct()
  .count())

+-----------------+
|DistinctCallTypes|
+-----------------+
|               30|
+-----------------+



30

In [6]:
# Example
few_fire_df = (fire_df
  .select("IncidentNumber", "AvailableDtTm", "CallType") 
  .where(col("CallType") != "Medical Incident"))
few_fire_df.show(5, truncate=False, vertical=True)

-RECORD 0--------------------------------
 IncidentNumber | 2003235                
 AvailableDtTm  | 01/11/2002 01:51:44 AM 
 CallType       | Structure Fire         
-RECORD 1--------------------------------
 IncidentNumber | 2003250                
 AvailableDtTm  | 01/11/2002 04:16:46 AM 
 CallType       | Vehicle Fire           
-RECORD 2--------------------------------
 IncidentNumber | 2003259                
 AvailableDtTm  | 01/11/2002 06:01:58 AM 
 CallType       | Alarms                 
-RECORD 3--------------------------------
 IncidentNumber | 2003279                
 AvailableDtTm  | 01/11/2002 08:03:26 AM 
 CallType       | Structure Fire         
-RECORD 4--------------------------------
 IncidentNumber | 2003301                
 AvailableDtTm  | 01/11/2002 09:46:44 AM 
 CallType       | Alarms                 
only showing top 5 rows



In [7]:
# filter for only distinct non-null CallTypes from all the rows
(fire_df
  .select("CallType")
  .where(col("CallType").isNotNull())
  .distinct()
  .show(10, False))

+-----------------------------+
|CallType                     |
+-----------------------------+
|Elevator / Escalator Rescue  |
|Aircraft Emergency           |
|Alarms                       |
|Odor (Strange / Unknown)     |
|Citizen Assist / Service Call|
|HazMat                       |
|Explosion                    |
|Oil Spill                    |
|Vehicle Fire                 |
|Suspicious Package           |
+-----------------------------+
only showing top 10 rows



In [8]:
# Rename column
new_fire_df = fire_df.withColumnRenamed("Delay", "ResponseDelayedinMins")
(new_fire_df
  .select("ResponseDelayedinMins")
  .where(F.col("ResponseDelayedinMins") > 5)
  .show(5, False))

+---------------------+
|ResponseDelayedinMins|
+---------------------+
|5.35                 |
|6.25                 |
|5.2                  |
|5.6                  |
|7.25                 |
+---------------------+
only showing top 5 rows



In [9]:
# date operations
fire_ts_df = (new_fire_df
  .withColumn("IncidentDate", F.to_timestamp(F.col("CallDate"), "MM/dd/yyyy"))
  .drop("CallDate") 
  .withColumn("OnWatchDate", F.to_timestamp(F.col("WatchDate"), "MM/dd/yyyy"))
  .drop("WatchDate") 
  .withColumn("AvailableDtTS", F.to_timestamp(F.col("AvailableDtTm"), 
  "MM/dd/yyyy hh:mm:ss a"))
  .drop("AvailableDtTm")
  # .show(5, truncate=False, vertical=True)
             )
# Select the converted columns
(fire_ts_df
  .select("IncidentDate", "OnWatchDate", "AvailableDtTS")
  .show(5, False))

+-------------------+-------------------+-------------------+
|IncidentDate       |OnWatchDate        |AvailableDtTS      |
+-------------------+-------------------+-------------------+
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:51:44|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 03:01:18|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 02:39:50|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 04:16:46|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 06:01:58|
+-------------------+-------------------+-------------------+
only showing top 5 rows



In [10]:
# what were the most common types of fire calls?
(fire_ts_df
  .select("CallType")
  .where(F.col("CallType").isNotNull())
  .groupBy("CallType")
  .count()
  .orderBy("count", ascending=False)
  .show(n=10, truncate=False))

+-------------------------------+------+
|CallType                       |count |
+-------------------------------+------+
|Medical Incident               |113794|
|Structure Fire                 |23319 |
|Alarms                         |19406 |
|Traffic Collision              |7013  |
|Citizen Assist / Service Call  |2524  |
|Other                          |2166  |
|Outside Fire                   |2094  |
|Vehicle Fire                   |854   |
|Gas Leak (Natural and LP Gases)|764   |
|Water Rescue                   |755   |
+-------------------------------+------+
only showing top 10 rows



In [11]:
# min,max, avg
(fire_ts_df
  .select(F.sum("NumAlarms"), F.avg("ResponseDelayedinMins"),
    F.min("ResponseDelayedinMins"), F.max("ResponseDelayedinMins"))
  .show())

+--------------+--------------------------+--------------------------+--------------------------+
|sum(NumAlarms)|avg(ResponseDelayedinMins)|min(ResponseDelayedinMins)|max(ResponseDelayedinMins)|
+--------------+--------------------------+--------------------------+--------------------------+
|        176170|         3.892364154521585|               0.016666668|                   1844.55|
+--------------+--------------------------+--------------------------+--------------------------+



In [12]:
# Q-1) How many distinct types of calls were made to the Fire Department?
# To be sure, let's not count "null" strings in that column.
(fire_df
  .select("CallType")
  .where(F.col("CallType").isNotNull())
  .distinct()
  .count())
# few_fire_df.show(20, vertical=False)

30

In [13]:
# Q-2) What are distinct types of calls were made to the Fire Department?
# These are all the distinct type of call to the SF Fire Department
(fire_df
  .select("CallType")
  .where(
      (F.col("CallType").isNotNull())
     & (F.col("City") == "SF")
  )
  .distinct()
  .count())

30

In [14]:
# Q-3) Find out all response or delayed times greater than 5 mins?
# Rename the column Delay - > ReponseDelayedinMins
# Returns a new DataFrame
# Find out all calls where the response time to the fire site was delayed for more than 5 mins
# new_fire_df = fire_df.withColumnRenamed("Delay", "ResponseDelayedinMins")
(fire_df
  .withColumn("OnWatchDate", F.col("Delay"))
  .select("Delay")
  .where(F.col("Delay") > 5)
  .count()
)

27508

In [15]:
# Q-4a) What zip codes accounted for most common calls?
# Let's investigate what zip codes in San Francisco accounted for most fire calls and what type where they.
# Filter out by CallType
# Group them by CallType and Zip code
# Count them and display them in descending order
# It seems like the most common calls were all related to Medical Incident, and the two zip codes are 94102 and 94103.
(fire_ts_df
  .select("Zipcode", "CallType")
  .where(F.col("Zipcode").isNotNull())
  .where(F.col("CallType").isNotNull())
  .groupBy("Zipcode", "CallType")
  .count()
  .orderBy("count", ascending=False)
  .show(n=10, truncate=False))

+-------+----------------+-----+
|Zipcode|CallType        |count|
+-------+----------------+-----+
|94102  |Medical Incident|16130|
|94103  |Medical Incident|14775|
|94110  |Medical Incident|9995 |
|94109  |Medical Incident|9479 |
|94124  |Medical Incident|5885 |
|94112  |Medical Incident|5630 |
|94115  |Medical Incident|4785 |
|94122  |Medical Incident|4323 |
|94107  |Medical Incident|4284 |
|94133  |Medical Incident|3977 |
+-------+----------------+-----+
only showing top 10 rows



In [16]:
# Q-4b) What San Francisco neighborhoods are in the zip codes 94102 and 94103
# Let's find out the neighborhoods associated with these two zip codes. In all likelihood, these are some of the contested neighborhood with high reported crimes.
(fire_ts_df
  .select("Neighborhood", "Zipcode")
  .where(F.col("Zipcode").isNotNull())
  .where((F.col("Zipcode")== 94102) | (F.col("Zipcode")== 94103))
  .count()
  # .show(n=10, truncate=False)
)

42737

In [17]:
# Q-5) What was the sum of all calls, average, min and max of the response times for calls?
# Let's use the built-in Spark SQL functions to compute the sum, avg, min, and max of few columns:
# Number of Total Alarms
# What were the min and max the delay in response time before the Fire Dept arrived at the scene of the call
(fire_df
  .select(F.sum("NumAlarms"), F.avg("Delay"),
    F.min("Delay"), F.max("Delay"))
  .show())

+--------------+-----------------+-----------+----------+
|sum(NumAlarms)|       avg(Delay)| min(Delay)|max(Delay)|
+--------------+-----------------+-----------+----------+
|        176170|3.892364154521585|0.016666668|   1844.55|
+--------------+-----------------+-----------+----------+



In [18]:
# ** Q-6b) What week of the year in 2018 had the most fire calls?**
# Note: Week 1 is the New Years' week and week 25 is the July 4 the week. Loads of fireworks, so it makes sense the higher number of calls.
# date operations
fire_ts_df_1 = (new_fire_df
  .withColumn("IncidentDate", F.to_timestamp(F.col("CallDate"), "MM/dd/yyyy"))
  .drop("CallDate") 
  .withColumn("OnWatchDate", F.to_timestamp(F.col("WatchDate"), "MM/dd/yyyy"))
  .drop("WatchDate") 
  .withColumn("AvailableDtTS", F.to_timestamp(F.col("AvailableDtTm"), 
  "MM/dd/yyyy hh:mm:ss a"))
  .drop("AvailableDtTm")
  # .show(5, truncate=False, vertical=True)
             )
(fire_ts_df_1
 .select(F.weekofyear("IncidentDate"), F.year("IncidentDate"))
 .where(F.year("IncidentDate") == 2018)
 .groupBy("weekofyear(IncidentDate)", "year(IncidentDate)")
 .count()
 .orderBy("count", ascending=False)
 .show(5, truncate=False, vertical=True)
)

-RECORD 0------------------------
 weekofyear(IncidentDate) | 22   
 year(IncidentDate)       | 2018 
 count                    | 259  
-RECORD 1------------------------
 weekofyear(IncidentDate) | 40   
 year(IncidentDate)       | 2018 
 count                    | 255  
-RECORD 2------------------------
 weekofyear(IncidentDate) | 43   
 year(IncidentDate)       | 2018 
 count                    | 250  
-RECORD 3------------------------
 weekofyear(IncidentDate) | 25   
 year(IncidentDate)       | 2018 
 count                    | 249  
-RECORD 4------------------------
 weekofyear(IncidentDate) | 1    
 year(IncidentDate)       | 2018 
 count                    | 246  
only showing top 5 rows



In [37]:
# ** Q-7) What neighborhoods in San Francisco had the worst response time in 2018?**
# It appears that if you living in Presidio Heights, the Fire Dept arrived in less than 3 mins, while Mission Bay took more than 6 mins.
q_7 = (fire_df
 .withColumn("IncidentDate", F.to_timestamp(F.col("CallDate"), "MM/dd/yyyy"))
 .select("City", F.year("IncidentDate"), "Neighborhood", "Delay")
 .where("City = 'San Francisco' AND YEAR(IncidentDate) = 2018")
 .groupBy("City", "Neighborhood")
 # .count()
 # .distinct()
 .agg(F.avg("Delay"))
 .orderBy("avg(Delay)", ascending=False)
 .show(10, truncate=False, vertical=True)
)

-RECORD 0--------------------------------------
 City         | San Francisco                  
 Neighborhood | Treasure Island                
 avg(Delay)   | 11.320833250880241             
-RECORD 1--------------------------------------
 City         | San Francisco                  
 Neighborhood | Presidio                       
 avg(Delay)   | 6.248148073752721              
-RECORD 2--------------------------------------
 City         | San Francisco                  
 Neighborhood | Chinatown                      
 avg(Delay)   | 6.158818309742307              
-RECORD 3--------------------------------------
 City         | San Francisco                  
 Neighborhood | McLaren Park                   
 avg(Delay)   | 4.74404764175415               
-RECORD 4--------------------------------------
 City         | San Francisco                  
 Neighborhood | Bayview Hunters Point          
 avg(Delay)   | 4.629759605458373              
-RECORD 5-------------------------------

In [46]:
# ** Q-8a) How can we use Parquet files or SQL table to store data and read it back?**
q_8 = (fire_df
 .withColumn("IncidentDate", F.to_timestamp(F.col("CallDate"), "MM/dd/yyyy"))
 .select("City", F.year("IncidentDate"), "Neighborhood", "Delay")
 .where("City = 'San Francisco' AND YEAR(IncidentDate) = 2018")
 .groupBy("City", "Neighborhood")
 # .count()
 # .distinct()
 .agg(F.avg("Delay"))
 .orderBy("avg(Delay)", ascending=False)
)

import tempfile
with tempfile.TemporaryDirectory(prefix="parquet") as d:
    # Write a DataFrame into a Parquet file
    q_8.write.parquet(d, mode="overwrite")

# ** Q-8c) How can read data from Parquet file?**
# Note we don't have to specify the schema here since it's stored as part of the Parquet metadata
    # Read the Parquet file as a DataFrame.
    q_8_c = spark.read.format("parquet").load(d)
    q_8_c.show()

+-------------+--------------------+------------------+
|         City|        Neighborhood|        avg(Delay)|
+-------------+--------------------+------------------+
|San Francisco|     Treasure Island|11.320833250880241|
|San Francisco|            Presidio| 6.248148073752721|
|San Francisco|           Chinatown| 6.158818309742307|
|San Francisco|        McLaren Park|  4.74404764175415|
|San Francisco|Bayview Hunters P...| 4.629759605458373|
|San Francisco|    Presidio Heights| 4.594131482319093|
|San Francisco|        Inner Sunset| 4.438095217981896|
|San Francisco|      Inner Richmond| 4.378439182090381|
|San Francisco|Financial Distric...| 4.340903129994245|
|San Francisco|      Haight Ashbury| 4.266428579390049|
|San Francisco|            Seacliff|  4.26111114025116|
|San Francisco|  West of Twin Peaks| 4.190952391851516|
|San Francisco|        Potrero Hill| 4.185406721094578|
|San Francisco|     Pacific Heights| 4.180453730971402|
|San Francisco|          Tenderloin| 4.101519513